## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

 For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

### Notes
 Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
 I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
 It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
 There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

for column in ['NFL', 'MLB', 'NBA', 'NHL']:
    df[column] = df[column].str.replace(r"\[.*\]", "")
    df[column] = df[column].apply(clean_team_name)


def get_nhl_data():
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    df = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    df.rename(columns={"Population (2016 est.)[8]" : "Population"}, inplace=True)
    df = clean_team_names(df)

    nhl_populations = df[['Metropolitan area', 'Population', 'NHL']].copy()
    nhl_populations['NHL'].replace(['—', ''], np.nan, inplace=True)
    nhl_populations.dropna(subset=['NHL'], inplace=True)
    nhl_populations['NHL'] = nhl_populations['NHL'].str.replace('[\w.]*\ ',"")
    nhl_populations['NHL'] = nhl_populations['NHL'].replace(['RangersIslandersDevils'],'Devils')
    nhl_populations.rename(columns={"NHL": "team"}, inplace=True)

    nhl_data = pd.read_csv('assets/nhl.csv')
    nhl_data['team'] = nhl_data['team'].str.replace('[*]', '')
    nhl_data = nhl_data[(nhl_data['year'] == 2018) & ~nhl_data.index.isin([0, 9, 18, 26])]
    nhl_data['W/L%'] = nhl_data['W'].astype(float) / (nhl_data['W'].astype(float) + nhl_data['L'].astype(float))

    teams_to_combine = {
        'Rangers': ['Rangers', 'Islanders', 'Devils'],
        'Kings': ['Kings', 'Ducks']
    }

    for combined_team, team_list in teams_to_combine.items():
        combined_wins = nhl_data[nhl_data['team'].isin(team_list)]['W'].sum()
        combined_losses = nhl_data[nhl_data['team'].isin(team_list)]['L'].sum()
        combined_ratio = combined_wins / (combined_wins + combined_losses)
        new_row = {
            'team': combined_team,
            'W': combined_wins,
            'L': combined_losses,
            'year': 2018,
            'W/L%': combined_ratio
        }
        nhl_data = nhl_data.append(new_row, ignore_index=True)

    merge = pd.merge(nhl_data, nhl_populations, how='outer', on='team').drop(28)
    merge = merge.astype({'W': int, 'L': int, 'Population': float, 'W/L%': float})
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population']
    win_loss_by_region = merge['W/L%']

    assert len(population_by_region) == len(win_loss_by_region) == 28, "Q1: There should be 28 teams being analyzed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


#For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.


In [ ]:
def get_nba_data():
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    df = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    df.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    df = clean_team_names(df)

    nba_populations = df[['Metropolitan area', 'Population', 'NBA']].copy()
    nba_populations['NBA'].replace(['—', ''], np.nan, inplace=True)
    nba_populations.dropna(subset=['NBA'], inplace=True)
    nba_populations['NBA'] = nba_populations['NBA'].str.replace('[\w.]*\ ',"")
    nba_populations.rename(columns={"NBA": "team"}, inplace=True)

    nba_data = pd.read_csv('assets/nba.csv')
    nba_data = nba_data[(nba_data['year'] == 2018)]
    nba_data['W/L%'] = nba_data['W'].astype(float) / (nba_data['W'].astype(float) + nba_data['L'].astype(float))

    teams_to_combine = {
        'Lakers': ['Lakers', 'Clippers'],
        'Knicks': ['Knicks', 'Nets']
        # Add more combined teams as needed
    }

    # Combine teams and calculate new win/loss ratios

    merge = pd.merge(nba_data, nba_populations, how='outer', on='team').drop(30)
    merge = merge.astype({'W': int, 'L': int, 'Population': float, 'W/L%': float})
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population']
    win_loss_by_region = merge['W/L%']

    assert len(population_by_region) == len(win_loss_by_region) == 28, "Q2: There should be 28 teams being analyzed for NBA"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


#For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.


In [ ]:
def get_mlb_data():
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    df = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    df.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    df = clean_team_names(df)

    mlb_populations = df[['Metropolitan area', 'Population', 'MLB']].copy()
    mlb_populations['MLB'].replace(['—', ''], np.nan, inplace=True)
    mlb_populations.dropna(subset=['MLB'], inplace=True)
    mlb_populations['MLB'] = mlb_populations['MLB'].str.replace('[\w.]*\ ',"")
    mlb_populations.rename(columns={"MLB": "team"}, inplace=True)

    mlb_data = pd.read_csv('assets/mlb.csv')
    mlb_data = mlb_data[(mlb_data['year'] == 2018)]
    mlb_data['W/L%'] = mlb_data['W'].astype(float) / (mlb_data['W'].astype(float) + mlb_data['L'].astype(float))

    teams_to_combine = {
        'GiantsAthletics': ['Giants', 'Athletics'],
        'DodgersAngels': ['Dodgers', 'Angels']
        # Add more combined teams as needed
    }

    # Combine teams and calculate new win/loss ratios

    merge = pd.merge(mlb_data, mlb_populations, how='outer', on='team').drop(30)
    merge = merge.astype({'W': int, 'L': int, 'Population': float, 'W/L%': float})
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population']
    win_loss_by_region = merge['W/L%']

    assert len(population_by_region) == len(win_loss_by_region) == 26, "Q3: There should be 26 teams being analyzed for MLB"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

#For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.


In [ ]:
def get_nfl_data():
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    df = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    df.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    df = clean_team_names(df)

    nfl_populations = df[['Metropolitan area', 'Population', 'NFL']].copy()
    nfl_populations['NFL'].replace(['—', ''], np.nan, inplace=True)
    nfl_populations.dropna(subset=['NFL'], inplace=True)
    nfl_populations['NFL'] = nfl_populations['NFL'].str.replace('[\w.]*\ ',"")
    nfl_populations.rename(columns={"NFL": "team"}, inplace=True)

    nfl_data = pd.read_csv('assets/nfl.csv')
    nfl_data = nfl_data[(nfl_data['year'] == 2018)]
    nfl_data['W/L%'] = nfl_data['W'].astype(float) / (nfl_data['W'].astype(float) + nfl_data['L'].astype(float))

    # Combine teams and calculate new win/loss ratios
    teams_to_combine = {
        'JetsGiants': ['Jets', 'Giants'],
        # Add more combined teams as needed
    }

    merge = pd.merge(nfl_data, nfl_populations, how='outer', on='team').drop(30)
    merge = merge.astype({'W': int, 'L': int, 'Population': float, 'W/L%': float})
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population']
    win_loss_by_region = merge['W/L%']

    assert len(population_by_region) == len(win_loss_by_region) == 29, "Q4: There should be 29 teams being analyzed for NFL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
